# Week 5 and 6 - Assignment 6
#### Team 5

**Goal:** Classify new "test" documents using already classified "training" documents and perform a sentiment analysis.

For this project, the dataset used was from the CrowdFlower library (https://www.crowdflower.com/data-for-everyone/), using the "Twitter sentiment analysis: Self-driving cars" dataset. This dataset includes posts from Twitter regarding users' opinions of self-driving cars, the tweets' classification as relevant to the topic or not, and if relevant the classification of tweet sentiment as integers (from a positive 5 to a negative 1).

# Importing data

#### Seeing what we're working with

In [205]:
import pandas as pd
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [206]:
data = pd.io.parsers.read_csv("https://raw.githubusercontent.com/Galanopoulog/DATA620-Assignment-6/master/Self-Driving-Cars.csv")
data = data.drop(data.columns[[1,2,3,4,7,8,9]], axis=1)
data.text.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True) #remove ascii characters from text
data.head(11)

,_unit_id,sentiment,sentiment:confidence,text
0,724227031,5,0.7579,Two places I'd invest all my money if I could:...
1,724227032,5,0.8775,Awesome! Google driverless cars will help the ...
2,724227033,2,0.6805,If Google maps can't keep up with road constru...
3,724227034,2,0.8820,Autonomous cars seem way overhyped given the t...
4,724227035,3,1.0000,Just saw Google self-driving car on I-34. It w...
5,724227036,3,1.0000,Will driverless cars eventually replace taxi d...
6,724227037,not_relevant,0.5367,Chicago metro expected to be fully autonomous ...
7,724227038,not_relevant,0.6548,I love the infotainment system in my new car. ...
8,724227039,5,0.7187,Autonomous vehicles could reduce traffic fatal...
9,724227040,1,0.6412,Driverless cars are not worth the risk. Don't...


In [207]:
len(data)

7156

In [208]:
print(data.dtypes)

In [209]:
# Count relevant and irrelevant
rel_count = len(data[data.sentiment!="not_relevant"])
irrel_count = len(data[data.sentiment=="not_relevant"])
neut_count = len(data[data.sentiment=="3"])

print "Relevant: %d" %rel_count
print "Not_Relevant: %d" %irrel_count
print "Neutral: %d" %neut_count

Two key things to note is that 1) the tweets and sentiment values are objects, which may need to be changed in the future for manipulations and 2) the number of non-relevant tweets is significantly smaller than the relevant portion, so splitting the data into training and test sets should probably be more methodical than just randomly shuffling the data. Also, it will be interesting to see the difference between the relevant neutral tweets and the irrelevant ones.

#### Splitting the data

The data is split into the training set, the evaluation set and the final set that will be used for testing.

In [210]:
# What sizes should our sets be?
train_set = int(0.7 * len(data))
dev_set = int(.15 * len(data))
final_set = len(data) - train_set - dev_set
Total = train_set + dev_set + final_set

print "Training set: %d" %train_set
print "Evaluation set: %d" %dev_set
print "Testing set: %d" %final_set
print "Total: %d" %Total

In [211]:
# Let's split our data into relevant and non-relevant parts.
relevant = data[data.sentiment!="not_relevant"]
nonrelevant = data[data.sentiment=="not_relevant"]

# Let's take 70%, 15% and the 15% of each
rel_train = int(0.7 * len(relevant))
rel_dev = int(.15 * len(relevant))
rel_fin = len(relevant) - rel_train - rel_dev
rel_Total = rel_train + rel_dev + rel_fin

irrel_train = int(0.7 * len(nonrelevant))
irrel_dev = int(.15 * len(nonrelevant))
irrel_fin = len(nonrelevant) - irrel_train - irrel_dev
irrel_Total = irrel_train + irrel_dev + irrel_fin

print "Training set: %d" %rel_train
print "Evaluation set: %d" %rel_dev
print "Testing set: %d" %rel_fin
print "Total: %d" %rel_Total
print "----------------------"
print "Training set: %d" %irrel_train
print "Evaluation set: %d" %irrel_dev
print "Testing set: %d" %irrel_fin
print "Total: %d" %irrel_Total

In [212]:
# Make sets for relevant data
rel_train = relevant[0:4860]
rel_dev = relevant[4860:5901]
rel_fin = relevant[5901:6943]

# Make sets for non-relevant data
irrel_train = nonrelevant[0:149]
irrel_dev = nonrelevant[149:180]
irrel_fin = nonrelevant[180:231]

# Combine sets by set types
train_set = pd.concat([rel_train, irrel_train], axis=0)
dev_set = pd.concat([rel_dev, irrel_dev], axis=0)
final_set = pd.concat([rel_fin, irrel_fin], axis=0)

# Determine lengths and compare to set size estimates
print "Training set: %d" %len(train_set)
print "Evaluation set: %d" %len(dev_set)
print "Testing set: %d" %len(final_set)

# Analyze and Categorize Data

In [213]:
from textblob.classifiers import NaiveBayesClassifier

In [214]:
subset = train_set[['text', 'sentiment']]
tuples = [tuple(x) for x in subset.values]

In [ ]:
from nltk.tokenize import word_tokenize
all_words = set(word.lower() for passage in tuples for word in word_tokenize(passage[0]))
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in tuples]

In [216]:
import nltk
classifier = nltk.NaiveBayesClassifier.train(t)
classifier.show_most_informative_features()

NameError: name 't' is not defined